In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pytpc
from pytpc.constants import *

In [ ]:
import random
import numpy as np
import pandas as pd
import math
import copy

In [ ]:
%load_ext Cython

In [ ]:
tilt = 7*degrees
clock = 6.25
shape = 332
vd_mag = -2.8

emag = 15e3
bmag = 0.5
ef, bf = pytpc.utilities.create_fields(emag, bmag, tilt)

vd = pytpc.simulation.drift_velocity_vector(vd_mag, emag, bmag, tilt)

pads = pytpc.generate_pad_plane()

In [ ]:
pt = pytpc.Particle(4, 2, 2, [0, 0, 1.], polar=pi-tilt, azimuth=3*pi/2)
gas = pytpc.gases.InterpolatedGasMixture(150., ('helium', 0.9), ('carbon_dioxide', 0.1))

In [ ]:
%%timeit
pt = pytpc.Particle(4., 2, 2, [0, 0, 1.], polar=pi-tilt, azimuth=3*pi/2)
simres = pytpc.track(pt, gas, ef, bf)

In [ ]:
pt.mass_num

In [ ]:
class pyobj(object):
    def __init__(self, param):
        self._param = param
        
    @property
    def param2(self):
        return self._param**2
    
    @param2.setter
    def param2(self, newval):
        self._param = newval**0.5
        
def pyfunc(n):
    p = pyobj(0)
    for i in range(n):
        p.param2 = i

In [ ]:
%%cython -a
cdef class cyobj:
    
    cdef int _param
    
    def __init__(self, int param):
        self._param = param
        
    property param2:
        def __get__(self):
            return self._param**2
    
        def __set__(self, int newval):
            self._param = newval**2
            
def cyfunc(int n):
    cdef cyobj c
    c = cyobj(0)
    cdef int i
    for i in range(n):
        c.param2 = i

In [ ]:
%%timeit 
pyfunc(10000)

In [ ]:
%%timeit 
c = cyfunc(10000)

In [ ]:
c = cyobj(4)

In [ ]:
%%timeit 
c.param2 = 5
c.param2

In [ ]:
a = np.array([4, 3, 2.])
p = pyobj(4)

In [ ]:
%%timeit 
p.param2 = 5 #np.random.uniform(0, 10, 3)
p.param2

In [ ]:
np.double